In [44]:
!pip install pandas matplotlib
!pip install -U kaleido


In [77]:
import pandas as pd
import networkx as nx
import plotly.graph_objects as go
import numpy as np

def visualize_chemical_interaction(file_path, output_path, top_n=300):
    # Load the data
    data = pd.read_csv(file_path)

    # Limit data to top_n entries
    data = data.head(top_n)

    # Create a directed graph
    G = nx.DiGraph()

    # Add edges to the graph
    for index, row in data.iterrows():
        chemical1 = row['Gene1']
        chemical2 = row['Gene2']
        interaction_type = row['Interaction type']
        regulation = row['Regulation']

        if pd.notna(chemical1) and pd.notna(chemical2):
            G.add_edge(chemical1, chemical2, interaction=f"{interaction_type}, {regulation}")

    # Create a layout for nodes
    pos = nx.spring_layout(G, k=0.5, seed=42)  # Using spring layout

    # Define edge colors based on interaction type
    edge_colors = {'positive': '#1f77b4', 'negative': '#ff7f0e', 'neutral': '#2ca02c'}

    # Create edge trace
    edge_trace = []
    for edge in G.edges(data=True):
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        interaction_details = edge[2]['interaction']
        interaction_type = interaction_details.split(',')[0].lower()
        edge_trace.append(
            go.Scatter(
                x=[x0, x1, None],
                y=[y0, y1, None],
                line=dict(width=2, color=edge_colors.get(interaction_type, '#333')),
                hoverinfo='text',
                text=[interaction_details],
                mode='lines'
            )
        )

    # Create node trace
    node_trace = go.Scatter(
        x=[],
        y=[],
        text=[],  # No direct text shown on nodes
        mode='markers',
        hoverinfo='text',
        marker=dict(
            showscale=True,
            colorscale='Viridis',
            color=[],
            size=10,  # Fixed size for small circles
            colorbar=dict(
                thickness=15,
                title='Node Degree',
                xanchor='left',
                titleside='right'
            ),
        )
    )

    for node in G.nodes():
        x, y = pos[node]
        degree = len(G[node])
        node_trace['x'] += tuple([x])
        node_trace['y'] += tuple([y])
        # Hover text includes the degree of the node
        node_trace['text'] += tuple([f"{node}<br>Degree: {degree}"])
        node_trace['marker']['color'] += tuple([degree])
        node_trace['marker']['size'] = [10] * len(G.nodes())  # Set fixed size for all nodes

    # Create the figure
    fig = go.Figure(
        data=edge_trace + [node_trace],
        layout=go.Layout(
            title='Gene-Gene Interaction Graph (Top 300)',
            titlefont_size=20,
            showlegend=False,
            hovermode='closest',
            margin=dict(b=20, l=20, r=20, t=40),
            xaxis=dict(showgrid=False, zeroline=False),
            yaxis=dict(showgrid=False, zeroline=False),
        ),
    )

    # Save the figure as an HTML file
    fig.write_html(output_path)

    # Show the figure
    fig.show()

# Replace 'path_to_your_file.csv' with your actual file path
file_path = '/content/Task110_Gene_to_Gene.csv'
output_path = '/content/Gene_Gene_interaction_graph.html'
visualize_chemical_interaction(file_path, output_path, top_n=300)
